In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import List

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
import numpy as np
import uproot
from scipy import stats

from HHbbVV.postprocessing import utils

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
MAIN_DIR = Path("../../../../")

plot_dir = MAIN_DIR / "plots/FTests/25Mar11nTFQCD11"
plot_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
def p_value(data_ts: float, toy_ts: list[float]):
    return np.mean(toy_ts >= data_ts)


def p_value_fit(data_ts: float, df: float):
    """get p-value from chi^2 it rather than toy values"""
    x = np.linspace(0.01, 500, 1000)
    cdf = stats.chi2.cdf(x, df)
    return 1 - cdf[utils.find_nearest(x, data_ts)]


def F_statistic(
    ts_low: list[float],
    ts_high: list[float],
    ord_low: int,
    ord_high: int,
    num_bins: int,
    dim: int,
):
    numerator = (ts_low - ts_high) / (ord_high - ord_low)
    denominator = ts_high / (num_bins - (ord_high + dim))

    return numerator / denominator

Nonresonant

In [ ]:
eos_cards_dir = "/eos/uscms/store/user/rkansal/bbVV/cards/f_tests/24Apr26NonresBDT995/ggf/"
local_cards_dir = "/uscms/home/rkansal/hhcombine/cards/f_tests/24Apr26NonresBDT995/ggf/"
# test_orders = [0, 1, 2, 3, 4, 5]
test_orders = [0, 1, 2]
test_statistics = {}
num_bins = 30

for o1 in test_orders:
    tdict = {"toys": {}, "data": {}, "ftoys": {}, "fdata": {}}
    tlabel = f"{o1}"

    for nTF in [o1, o1 + 1]:
        tflabel = f"{nTF}"

        # test statistics for toys generated by (o1, o2) order model
        file = uproot.concatenate(
            f"{eos_cards_dir}/nTF_{nTF}/higgsCombineToys{tlabel}Seed*4.GoodnessOfFit.mH125.*4.root"
        )
        tdict["toys"][tflabel] = np.array(file["limit"])

        # data test statistic
        file = uproot.concatenate(
            f"{local_cards_dir}/nTF_{nTF}/higgsCombineData.GoodnessOfFit.mH125.root"
        )
        tdict["data"][tflabel] = file["limit"][0]

        if nTF != o1:
            tdict["ftoys"][tflabel] = F_statistic(
                tdict["toys"][tlabel], tdict["toys"][tflabel], o1, nTF, num_bins, 1
            )
            tdict["fdata"][tflabel] = F_statistic(
                tdict["data"][tlabel], tdict["data"][tflabel], o1, nTF, num_bins, 1
            )

    test_statistics[tlabel] = tdict

Resonant

In [ ]:
# eos_cards_dir = "/eos/uscms/store/user/rkansal/bbVV/cards/f_tests/23Jul6WP06/"
# local_cards_dir = "/uscms/home/rkansal/hhcombine/cards/f_tests/23Jul6WP06/"
eos_cards_dir = "/eos/uscms/store/user/rkansal/bbVV/cards/f_tests/25Mar11nTFQCD11/"
local_cards_dir = "/uscms/home/rkansal/hhcombine/cards/f_tests/25Mar11nTFQCD11/"
test_orders = [(0, 0), (1, 0), (0, 1), (1, 1), (1, 2), (2, 1), (2, 2)]  # , (1, 2), (3, 1)]
# test_orders = [(0, 0)]
test_statistics = {}
num_bins = 10 * 14 * 2

for o1, o2 in test_orders:
    tdict = {"toys": {}, "data": {}, "ftoys": {}, "fdata": {}}
    tlabel = f"{o1}{o2}"
    print(tlabel)

    for nTF1, nTF2 in [(o1, o2), (o1 + 1, o2), (o1, o2 + 1)]:
        # for nTF1, nTF2 in [(o1, o2)]:
        tflabel = f"{nTF1}{nTF2}"

        # test statistics for toys generated by (o1, o2) order model
        file = uproot.concatenate(
            f"{eos_cards_dir}/nTF1_{nTF1}_nTF2_{nTF2}/higgsCombineToys{tlabel}.GoodnessOfFit.mH125.*9.root"
        )
        tdict["toys"][tflabel] = np.array(file["limit"])

        # data test statistic
        file = uproot.concatenate(
            f"{local_cards_dir}/nTF1_{nTF1}_nTF2_{nTF2}/higgsCombineData.GoodnessOfFit.mH125.root"
        )
        tdict["data"][tflabel] = file["limit"][0]

        if not (nTF1 == o1 and nTF2 == o2):
            tdict["ftoys"][tflabel] = F_statistic(
                tdict["toys"][f"{o1}{o2}"],
                tdict["toys"][tflabel],
                o1 + o2,
                nTF1 + nTF2,
                num_bins,
                2,
            )
            tdict["fdata"][tflabel] = F_statistic(
                tdict["data"][f"{o1}{o2}"],
                tdict["data"][tflabel],
                o1 + o2,
                nTF1 + nTF2,
                num_bins,
                2,
            )

    test_statistics[tlabel] = tdict

QCD F-tests

In [ ]:
cards_dir = "/home/users/rkansal/hhcombine/cards/qcdftests_readw2"
test_orders = [(0, 0), (1, 0), (0, 1), (1, 1), (2, 0), (0, 2), (2, 1), (1, 2), (3, 1)]
test_orders = [(1, 1)]
test_statistics = {}
num_bins = 10 * 14 * 2

for o1, o2 in test_orders:
    tdict = {"toys": {}, "data": {}, "ftoys": {}, "fdata": {}}
    tlabel = f"{o1}{o2}"
    print(tlabel)

    for nTF1, nTF2 in [(o1, o2), (o1 + 1, o2), (o1, o2 + 1), (o1 + 1, o2 + 1)]:
        tflabel = f"{nTF1}{nTF2}"

        # test statistics for toys generated by (o1, o2) order model
        file = uproot.concatenate(
            f"{cards_dir}/nTF{nTF1}{nTF2}/higgsCombineToys{tlabel}.GoodnessOfFit.mH125.*6.root"
        )
        tdict["toys"][tflabel] = np.array(file["limit"])

        # data test statistic
        file = uproot.concatenate(
            f"{cards_dir}/nTF{nTF1}{nTF2}/higgsCombineData.GoodnessOfFit.mH125.root"
        )
        tdict["data"][tflabel] = file["limit"][0]

        if not (nTF1 == o1 and nTF2 == o2):
            tdict["ftoys"][tflabel] = F_statistic(
                tdict["toys"][f"{o1}{o2}"],
                tdict["toys"][tflabel],
                o1 + o2,
                nTF1 + nTF2,
                num_bins,
                2,
            )
            tdict["fdata"][tflabel] = F_statistic(
                tdict["data"][f"{o1}{o2}"],
                tdict["data"][tflabel],
                o1 + o2,
                nTF1 + nTF2,
                num_bins,
                2,
            )

    test_statistics[tlabel] = tdict

In [ ]:
def plot_tests(
    data_ts: float,
    toy_ts: np.ndarray,
    name: str,
    title: str = None,
    bins: int = 15,
    fit: str = None,
    fdof2: int = None,
    xlim: float = None,
    plot_max: float = None,
):
    if plot_max is None:
        plot_max = max(np.max(toy_ts), data_ts)
    # plot_max = max(np.max(toy_ts), data_ts) if fit != "chi2" else 200
    # plot_min = min(np.min(toy_ts), data_ts, 0)
    plot_min = 0
    pval = p_value(data_ts, toy_ts)
    res = None

    plt.figure(figsize=(12, 8))
    h = plt.hist(
        toy_ts,
        np.linspace(plot_min, plot_max if xlim is None else xlim, bins + 1),
        color="#8C8C8C",
        histtype="step",
        label=f"{len(toy_ts)} Toys",
    )

    if fit is not None:
        x = np.linspace(plot_min + 0.01, plot_max, 100)

        if fit == "chi2":
            res = stats.fit(stats.chi2, toy_ts, [(0, 200)])
            pdf = stats.chi2.pdf(x, res.params.df)
            pval = p_value_fit(data_ts, res.params.df)
            label = rf"$\chi^2_{{DoF = {res.params.df:.2f}}}$ Fit"
        elif fit == "f":
            pdf = stats.f.pdf(x, 1, fdof2)
            label = rf"$F-dist_{{DoF = (1, {fdof2})}}$"
        else:
            raise ValueError("Invalid fit")

        plt.plot(
            x,
            pdf * (np.max(h[0]) / np.max(pdf)),
            color="#1f78b4",
            linestyle="--",
            # alpha=0.6,
            label=label,
        )

    plt.axvline(data_ts, color="#FF502E", linestyle=":", label=rf"Data ($p$-value = {pval:.2f})")

    hep.cms.label(
        "Work in Progress",
        data=True,
        lumi=138,
        year=None,
    )

    _ = plt.legend()
    plt.title(title)
    plt.ylabel("Number of Toys")
    plt.xlabel("Test Statistics")

    plt.savefig(f"{plot_dir}/{name}.pdf", bbox_inches="tight")

    return res

Nonresonant

In [ ]:
o1 = 2  # order being tested
tlabel = f"{o1}"

data_ts, toy_ts = test_statistics[tlabel]["data"][tlabel], test_statistics[tlabel]["toys"][tlabel]
res = plot_tests(data_ts, toy_ts, "gof" + tlabel, fit="chi2", bins=20)

ord1 = o1 + 1
tflabel = f"{ord1}"
data_ts, toy_ts = pval = (
    test_statistics[tlabel]["fdata"][tflabel],
    test_statistics[tlabel]["ftoys"][tflabel],
)
plot_tests(data_ts, toy_ts, f"f{tlabel}_{tflabel}", title=f"{o1} vs. {ord1}", xlim=100)

Resonant

In [ ]:
o1, o2 = 2, 1  # order being tested
tlabel = f"{o1}{o2}"

data_ts, toy_ts = test_statistics[tlabel]["data"][tlabel], test_statistics[tlabel]["toys"][tlabel]
plot_tests(data_ts, toy_ts, "gof" + tlabel, fit="chi2", bins=20)

for ord1, ord2 in [[o1 + 1, o2], [o1, o2 + 1]]:
    # for ord1, ord2 in [[o1 + 1, o2 + 1]]:
    tflabel = f"{ord1}{ord2}"
    data_ts, toy_ts = pval = (
        test_statistics[tlabel]["fdata"][tflabel],
        test_statistics[tlabel]["ftoys"][tflabel],
    )
    print("WARNING! TEMPORARILY SWITCHING ORD1 VS ORD2 IN PLOTS - REMEMBER TO CHANGE NEXT TIME!")
    plot_tests(
        data_ts,
        toy_ts,
        f"f{tlabel}_{tflabel}",
        # title=f"({o1}, {o2}) vs. ({ord1}, {ord2})",
        title=f"({o2}, {o1}) vs. ({ord2}, {ord1})",
        plot_max=25,
    )

In [ ]:
data_ts, toy_ts = tdict["data"][tlabel], tdict["toys"][tlabel]
plot_tests(data_ts, toy_ts, "gof" + tlabel, fit="chi2", bins=20)